
<!-- 크롤링은? 
	데이터가 있다!! 
	토큰으로 쪼개서 영어로 변역해서 huggingface로 넣고 돌려 봐
	결과가 나와 그게 실제와 맞는지 확인해봐

	크롤링도 할 수 있음 해보는게 좋다. 
	 -->

In [14]:
# !pip install requests

In [15]:
import pandas as pd 
import re
import tensorflow as tf
import requests
import urllib.parse
import pandas as pd
import json

### 크롤링 나온 csv 토큰화 시키기

In [16]:
data = pd.read_csv("./NewsData/반도체_20240628_17시32분.csv")
data.head()

,date,title,link,content
0,2024-06-28 03:35:21,"한미 반도체 포럼 개최…""반도체 협력이 한미동맹의 핵심""(종합)",https://n.news.naver.com/mnews/article/001/001...,"[\n삼성·SK·인텔·IBM 등 참여…안덕근 산업장관 ""양국 기술협력 지원""美상무장..."
1,2024-06-28 08:57:43,"한·미반도체협회, 협력 MOU 체결…공급망 협력 강화",https://n.news.naver.com/mnews/article/003/001...,"[\n삼성·SK하닉·인텔·퀄컴 등 한자리협회, 'AI반도체 혁신센터' 美구축\n\n..."
2,2024-06-28 05:10:00,"정부 ""바이오는 제2의 반도체""… 특화단지 지정해 키운다",https://n.news.naver.com/mnews/article/417/000...,[\n인천·경기 등 5곳 선정높은 성장성 겨냥개발·제조 생태계 구축\n\n\n\n정...
3,2024-06-28 15:53:13,"SK하이닉스 ""AI반도체 리더십 굳힌다""…고성능 SSD 개발",https://n.news.naver.com/mnews/article/015/000...,[\n\n\n\n\nSK하이닉스가 온디바이스(내장형) 인공지능(AI) PC에 탑재되...
4,2024-06-28 06:00:00,"SK, 1박2일 전략회의 돌입…""배터리·바이오보다 AI·반도체""",https://n.news.naver.com/mnews/article/003/001...,[\n28~29일 이천 SKMS연구소 개최최재원·창원 등 경영진 30여명 참석美출장...


In [17]:
data['content'][0]

'[\n삼성·SK·인텔·IBM 등 참여…안덕근 산업장관 "양국 기술협력 지원"美상무장관 "AI 때문에 반도체 더 중요해져…미국 생산 필요하다"\n\n\n\n한미 공급망·산업 대화(SCCD) 반도체 포럼(워싱턴=연합뉴스) 김동현 특파원 = 한국반도체산업협회 김정회 상근부회장과 존 뉴퍼 미국반도체산업협회 회장이 27일(현지시간) 워싱턴DC의 한 호텔에서 열린 \'한미 공급망·산업 대화(SCCD) 반도체 포럼\'에서 반도체 공급망 협력을 위한 양해각서(MOU)를 체결하고 있다. 안덕근 산업통상자원부 장관과 지나 러몬도 미국 상무부 장관이 서명식을 지켜보고 있다. 2024.6.27(세종·워싱턴=연합뉴스) 차대운 기자 김동현 특파원 = 한미 양국간 반도체 산업 분야의 협력을 강화하기 위한 \'한미 공급망·산업 대화(SCCD) 반도체 포럼\'이 27일(현지시간) 미국 워싱턴DC에서 개최됐다.     한미 산업장관이 주관하는 첨단산업 및 공급망 분야 정례 협의채널인 SCCD 개최 계기에 양국 반도체협회 주최로 처음 열린 민관 포럼에는 안덕근 산업부 장관, 지나 러몬도 미국 상무부 장관 등 양국 정부 인사와 삼성전자, SK하이닉스, 인텔, IBM, 시놉시스 등 양국 반도체 기업 관계자들이 참석했다.    참석자들은 양국 반도체 산업의 현안인 ▲ 기술 개발 ▲ 인력 양성 ▲ 공급망 안정 등 현황을 점검하고 각 분야에서 양국 간 협력 강화 방안을 논의했다.    양국 반도체 협회는 반도체 공급망 협력을 위한 양해각서(MOU)를 체결, 반도체 포럼 정례화와 인공지능(AI) 등 신흥 시장에서의 비즈니스 협력, 기술 개발, 인력 양성, 투자 활성화 등 협력을 추진하기로 했다고 산업통상자원부는 밝혔다.    안 장관은 이날 포럼에서 가전과 휴대전화뿐만 아니라 방위 산업에서도 반도체가 필요하다면서 "한미 안보 동맹이 공급망과 산업, 과학기술을 아우르는 동맹으로 진화하는 과정에서 반도체 협력이 우리 양자 동맹의 핵심 기둥(pillar)을 구성한다고 믿는다"고 말했다.     안 장관은 "

In [62]:
# 데이터 로드 및 정제하여 df에 정제된 content 추가하기 
def data_load(filename) :

    from pprint import pprint
    import json
    import os

    from openai import OpenAI


    data = pd.read_csv("./NewsData/"+filename)
    data2 = data.copy() # data2에 data 복제
    newContent = []

    # for문 돌리면서 기호 제거 
    for i in data2['content'] :
        newContent.append(pad_punctuation(i))


    # 일단은 df에 제거된 newContent리스트 추가 
    data2['newContent'] = newContent
    data2.head()

    data2['newContent'][0]

    sample = data2['newContent']


    api_key= os.environ.get("OPENAI_API_KEY")
    client = OpenAI(api_key=api_key)


    template = """
    당신은 번역함수이며, 반환값은 반드시 JSON 데이터 여야 합니다. 
    STEP 별로 작업을 수행하면서 그 결과를 아래의 출력 결과 JSON 포맷에 작성하세요.
    STEP-1. 입력받은 텍스트가 긍정적이라면 True를 표기할것
    STEP-2. STEP-1의 결과가 왜 그런지 이유를 표기할 것. 
    ```{text}```
    ---
    출력결과 : {{"STEP-1":<true/false>, "STEP-2": <이유> }}
    """
    for text in sample:
        current_template = template.format(text=text)
        context = [{"role": "user", "content": current_template}]

        model_1 = 'gpt-3.5-turbo-1106'
        model_2 = 'gpt-4-0613'
        try:
            response = client.chat.completions.create(
                model=model_1,
                messages=context,
                temperature=0,
                top_p=0,
                seed=1234
            ).model_dump()
            parsed_response = json.loads(response['choices'][0]['message']['content'])
            pprint(parsed_response)
        except Exception as e:
            print(f"Error processing text: {text}")
            print(e)


# \n, \t 불필요한 내용 제거를 위한 함수
def pad_punctuation(s):
    s = re.sub(r'\n', '', s)
    s = re.sub(r'\t', '', s)
    s = re.sub(r'\[', '', s)
    s = re.sub(r'\]', '', s)
    s = re.sub(r'\=', '', s)
    s = re.sub(r'\\', '', s)
    s = re.sub(r'\(', '', s)
    s = re.sub(r'\)', '', s)
    s = re.sub(r'', '', s)
    s = re.sub(r"[\'\"]", '', s)
    
    return s



In [63]:
data_load("반도체_20240704_10시12분.csv")

Error processing text: 美 10년물 국채 0.08%p 넘게 하락…기준금리 인하 기대감 높여엔비디아 로고로이터 연합뉴스 자료사진. 재판매 및 DB 금지샌프란시스코연합뉴스 김태종 특파원  미국 중앙은행인 연방준비제도Fed·연준의 기준금리 인하 가능성을 높여주는 지표가 나오면서 3일현지시간 뉴욕 증시에서 반도체주들이 일제히 상승 마감했다.     인공지능AI 칩 선두 주자 엔비디아의 이날 주가는 전날보다 4.57% 오른 128.28달러17만7천860원에 거래를 마쳤다.    이날 뉴욕 증시는 연방 공휴일인 미국 독립기념일4일 휴장을 하루 앞두고 거래시간이 오후 1시미 동부 시간 기준로 단축됐다.    엔비디아 주가는 이날 소폭 하락 출발했으나, 이후 상승 전환한 뒤 그 폭을 확대하며 130달러에 다가섰다.     시가총액도 3조1천550억 달러로 불어나 3위 자리를 지키며 2위 애플+0.58·3조3천970억 달러과의 격차를 좁혔다.    10대 1의 주식 분할을 앞둔 미 반도체 기업 브로드컴 주가는 4.33% 올랐고, 마이크론 테크놀로지와 대만 반도체 기업 TSMC도 각각 3.19%와 2.19% 상승 마감했다.        퀄컴의 주가도 전날보다 1.82% 올랐다. AMD는 소폭0.25% 하락 마감했다.     주요 반도체 기업 주가가 대부분 상승하면서 필라델피아 반도체 지수도 1.92% 오른 가운데 거래를 마쳤다.    이들 반도체주 상승은 연준의 기준금리 인하 가능성에 힘을 보태는 지표들이 나온 데 따른 것으로 풀이된다.    미 고용정보업체 오토매틱데이터프로세싱ADP가 이날 발표한 6월 미국의 민간기업 고용 증가 폭전월 대비 15만명 증가은 4개월 만에 가장 작았고, 다우존스가 집계한 전문가 전망치16만명도 밑돌았다.    미 노동부가 발표한 6월 16∼22일 주간 계속 실업수당최소 2주 이상 실업수당 신청 청구 건수도 약 2년 7개월 만에 가장 높은 수준을 나타냈다.    연준은 기준금리 인하에 앞서 인플레이션 둔화세 지속에 대한 확신이 필

In [21]:
# !pip install --upgrade openai
# !pip install --upgrade httpx
# !pip install --upgrade httpx openai


In [64]:
# from openai import OpenAI

# api_key= os.environ.get("OPENAI_API_KEY")

# os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
# print(api_key)
